In [1]:
import torch
import numpy as np
import normflows as nf

from matplotlib import pyplot as plt
from tqdm import tqdm

/home/pgrad2/2448355h/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [33]:
K = 64
torch.manual_seed(0)

latent_size = 3
b = torch.Tensor([1 if i % 2 == 0 else 0 for i in range(latent_size)])
flows = []
for i in range(K):
    s = nf.nets.MLP([latent_size, 2 * latent_size, latent_size], init_zeros=True)
    t = nf.nets.MLP([latent_size, 2 * latent_size, latent_size], init_zeros=True)
    if i % 2 == 0:
        flows += [nf.flows.MaskedAffineFlow(b, t, s)]
    else:
        flows += [nf.flows.MaskedAffineFlow(1 - b, t, s)]
    flows += [nf.flows.ActNorm(latent_size)]

# Set target and q0
target = nf.distributions.TwoModes(2, 0.1)
q0 = nf.distributions.DiagGaussian(3)

# Construct flow model
nfm = nf.NormalizingFlow(q0=q0, flows=flows, p=target)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
nfm = nfm.to(device)
nfm = nfm.double()

# # Initialize ActNorm
# z, _ = nfm.sample(num_samples=2 ** 7)
# z_np = z.to('cpu').data.numpy()
# plt.figure(figsize=(15, 15))
# plt.hist2d(z_np[:, 0].flatten(), z_np[:, 1].flatten(), (200, 200), range=[[-3, 3], [-3, 3]])
# plt.gca().set_aspect('equal', 'box')
# plt.show()

In [26]:
nf.flows.ActNorm?

In [34]:
nfm.sample(num_samples=2 ** 7)

(tensor([[-0.2310, -0.2466,  0.2542],
         [-2.2561,  0.2985,  0.6747],
         [ 0.0110,  0.7697, -0.1960],
         [-0.3830,  0.3615,  1.4689],
         [-1.8308, -0.2434,  1.4231],
         [ 1.4952, -1.3010,  0.0448],
         [-1.4006, -1.5375,  0.8261],
         [-0.2512, -0.9926, -0.4794],
         [ 3.3470, -0.1726, -1.6598],
         [ 1.4229, -0.6594, -0.7005],
         [-1.3513, -0.3014, -0.1973],
         [-0.5242, -0.2385,  0.1218],
         [-0.6400, -0.1468,  1.4666],
         [-1.8371,  0.5541, -0.6326],
         [ 1.2369,  1.1436,  0.2803],
         [ 0.1570, -0.6331,  0.2227],
         [-0.1866, -1.0101,  0.7032],
         [-1.0066,  0.8973,  0.3347],
         [-0.2482, -1.0921,  0.6724],
         [-0.1055, -0.8088, -1.0538],
         [-1.4080, -1.0828, -1.4317],
         [ 0.0810, -0.0929,  0.9800],
         [-0.6054,  0.6620,  0.5119],
         [ 0.2031,  1.7305,  0.3904],
         [ 0.4066, -1.2152, -1.4456],
         [-0.8567, -1.2097, -0.0196],
         [ 1

In [23]:
nfm.forward(torch.tensor([1.0,20.0],device=device))

tensor([[nan, nan]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [3]:
nf.NormalizingFlow(q0=q0, flows=flows, p=target).forward(torch.tensor([1.0,20.0],device=device))

/home/pgrad2/2448355h/.local/lib/python3.10/site-packages/normflows/flows/normalization.py:23: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  s_init = -torch.log(z.std(dim=self.batch_dims, keepdim=True) + 1e-6)


tensor([[nan, nan]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [127]:
np.exp(np.max(samples,axis=1))

/tmp/ipykernel_2322888/1977296910.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(np.max(samples,axis=1))


array([inf, inf, inf, ..., inf, inf, inf])

In [128]:
np.log(np.exp(np.max(samples,axis=1)).mean())

/tmp/ipykernel_2322888/1160389807.py:1: RuntimeWarning: overflow encountered in exp
  np.log(np.exp(np.max(samples,axis=1)).mean())


inf

In [129]:
max_U = torch.tensor(np.max(samples,axis=1) ) # shape: (batch_size,)

# We want log( mean( exp(max_U) ) ) over the batch dimension
log_E_exp_max_U = log_mean_exp(max_U, dim=0)
log_E_exp_max_U

tensor(2028.5075, dtype=torch.float64)

In [130]:
U_samples = f_U_model.sample((num_U_samples,))
max_U = torch.max(U_samples, dim=1)[0]
E_exp_max_U = torch.mean(torch.exp(max_U))

max_U = torch.max(U_samples, dim=1)[0]

# 2) Shift and exponentiate
shifted_exp = torch.exp(x - x_max)

# 3) Average, then take log
mean_shifted_exp = torch.mean(shifted_exp, dim=dim, keepdim=True)
log_mean_shifted_exp = torch.log(mean_shifted_exp)

# 4) Add the max back
out = x_max + log_mean_shifted_exp

NameError: name 'dim' is not defined

In [3]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA RTX A6000 is available.


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [30]:
a = [1,2,3]
# a.pop()
print(a)

[1, 2, 3]


In [158]:
class Node(object):
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next
        
node1 = Node(1)
node2 = Node(2)
node3 = Node(3)
node4 = Node(4)
node5 = Node(5)

node1.next = node2
node2.next = node3
node3.next = node4
node4.next = node5

In [83]:
node_odd = node1

In [107]:
print(node4.next.val)

AttributeError: 'NoneType' object has no attribute 'val'

In [90]:
node_odd.next.val

13

In [157]:
def oddEvenList(head):
    """
    :type head: Optional[ListNode]
    :rtype: Optional[ListNode]
    """
    node_odd = head
    node_oven = head.next
    count = 1
    while node_odd.next and node_odd.next.next:
        node_odd.next = node_odd.next.next
        node_odd = node_odd.next
    while node_oven.next and node_oven.next.next:
        print(node_oven.val)
        print(node_oven.next.val)
        print(node_oven.next.next.val)
        node_oven.next = node_oven.next.next 
        if count == 1:
             node_oven_start = node_oven
        node_oven = node_oven.next
        print(node_oven.val)
        count += 1

    node_oven.next = None
    node_odd.next = node_oven_start
    return head

In [159]:
oddEvenList(node1)

2
3
5
5


In [147]:
head = node1
count = 1
while head and count<=10:
    print(head.val)
    head = head.next
    count += 1

1
3
5
2
5
2
5
2
5
2


In [49]:
" example   good a".split()

['example', 'good', 'a']